In [9]:
# pip install xlrd
# import xlrd
import pandas as pd
import numpy as np
import nltk 
import re, string, unicodedata
import sys
import urllib
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import word_tokenize, sent_tokenize
import urllib2  
nltk.download('wordnet')
nltk.download('stopwords')
import requests
from bs4 import BeautifulSoup
# import xlrd

def syllables(words):
    sum1=0
    for word in words:
        count = 0
#         vowels = 'aeiouy'
        if word.endswith(('es','ed')):
                for w in word:
                     if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                        count += 1
                if(count>3):
                     sum1+=sum1
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                        count += 1
            if(count>2):
                 sum1+=sum1
    return sum1



def remove_stopwords(words):
    stopset = set(w.upper() for w in stopwords.words('english'))
    new_words = []
    for word in words:
        if word not in stopset:
            new_words.append(word)
    return new_words



def normalize_making(words):
    new_words = []
    for word in words:
        new_word = word.encode('ascii', 'ignore').decode('ascii').strip()

        new_word = re.sub('\[[^]]*\]', '', new_word)
        new_word = re.sub('[\d%/$]', '', new_word)
        new_word = re.sub(r'[^\w\s]', '', new_word)
        if new_word != '':
            new_words.append(word.upper())
    return new_words

  

    
positive_negative = pd.read_csv('LoughranMcDonald_MasterDictionary_2016.csv', index_col= 0)
constraining = set(pd.read_excel('constraining_dictionary.xlsx',index_col = 0).index)
uncertainty = set(pd.read_excel('uncertainty_dictionary.xlsx', index_col = 0).index)
x1  = pd.ExcelFile('cik_list.xlsx')
df1 = x1.parse('cik_list_ajay')
section_name_excel = ['MDA','QQDMR','RF']
variables = ['positive_score','negative_score','polarity_score','average_sentence_length', 'percentage_of_complex_words',\
                   'fog_index','complex_word_count','word_count','uncertainty_score','constraining_score', 'positive_word_proportion',\
                   'negative_word_proportion', 'uncertainty_word_proportion', 'constraining_word_proportion']
df_col = []
for i in section_name_excel:
    for j in variables:
            k = i.lower() + '_' + j
            df_col.append(k)
    df2 = pd.DataFrame(columns=df_col)
    
    
    
max_row, max_col = df1.shape
df1['SECFNAME'] = df1['SECFNAME'].apply(lambda x: "{}{}".format('https://www.sec.gov/Archives/', x))
constraining_whole = pd.Series(name='constraining_whole')


########################################################################################

def main_function(str1):
    sentence = sent_tokenize(str1)
    sentence_length = len(sentence)
    
    sent_list = word_tokenize(str1)
    word_count = len(sent_list)
    uncertainty_score = 0 
    constraining_score= 0
    positive_score = 0
    negative_score = 0
    fog_index = 0
    percentage_of_complex_words = 0
    polarity_score = 0
    positive_word_proportion = 0 
    negative_word_proportion =  0 
    uncertainty_word_proportion = 0 
    constraining_word_proportion = 0 
    average_sentence_length=0
    complex_words=0
    
    if word_count!= 0 :
        average_sentence_length = word_count/sentence_length

        
        sent_list = remove_stopwords(sent_list)
        complex_words = syllables(sent_list)
        sent_list = normalize_making(sent_list)
        percentage_of_complex_words = complex_words/word_count
        fog_index = 0.4 * (average_sentence_length + percentage_of_complex_words)
        

        new_words = []
        sent_list = normalize_making(sent_list)

        for word in sent_list:
            if word in positive_negative.index:
                if positive_negative.loc[word].Positive > 0:
                        positive_score += 1
                if positive_negative.loc[word].Negative > 0:
                        negative_score += 1
            if word in uncertainty:
                        uncertainty_score += 1
            if word in constraining:
                        constraining_score += 1
        
        polarity_score = (positive_score-negative_score)/(positive_score + negative_score + .000001)
        positive_word_proportion = positive_score/word_count
        negative_word_proportion = negative_score/word_count
        uncertainty_word_proportion = uncertainty_score/word_count
        constraining_word_proportion = constraining_score/word_count
    variable_names = [positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,\
                     fog_index, complex_words, word_count, uncertainty_score,constraining_score, positive_word_proportion,\
                     negative_word_proportion, uncertainty_word_proportion, constraining_word_proportion]
    return variable_names



###################################################################################
    
for row in range(max_row):
    i = df1.loc[row]['SECFNAME']
    r = requests.get(i)    
    soup = BeautifulSoup(r.text, 'html.parser',from_encoding="utf-8")
    stripped_text = soup.get_text()


    with open('myfile1.txt', 'w+') as f:
        print(row)
        the_text = (stripped_text).encode('utf-8')
        file = f.write(the_text)
    
    
############################################################################################    
    
    recording = False
    output_section_mda = []
    for line in open('myfile1.txt').readlines():

        if recording is False:
            if re.search(r'ITEM [0-9]. MANAGEMENT\'S DISCUSSION AND ANALYSIS', line) is not None:
                recording = True
                output_section_mda.append(line.decode('utf-8').strip())
        elif recording is True:
            if re.search(r'ITEM [0-9].', line) is not None:
                recording = False
            output_section_mda.append(line.decode('utf-8').strip())
    
    
    

    str1 = ''.join(output_section_mda)
    variable_names = main_function(str1)
    
    
    
    
    df2.loc[row] = np.zeros(42)

    k="MDA"
    length=0
    for j in variables:
        col = k.lower() + '_' + j
        df2.loc[row][col] = variable_names[length]
        length+=1

        
#########################################################################################
    
    
    
    
    recording = False
    output_section_QQDMR = []
    for line in open('myfile1.txt').readlines():

        if recording is False:
            if re.search(r'ITEM [0-9]. Quantitative and Qualitative Disclosures about Market Risk', line) is not None:
                recording = True
                output_section_QQDMR.append(line.decode('utf-8').strip())
        elif recording is True:
            if re.search(r'ITEM [0-9].', line) is not None:
                recording = False
            output_section_QQDMR.append(line.decode('utf-8').strip())
    str1 = ''.join(output_section_QQDMR)

    
    variable_names = main_function(str1)

    k="QQDMR"
    length=0
    for j in variables:
        col = k.lower() + '_' + j
        df2.loc[row][col] = variable_names[length]
        length+=1
    
    
#########################################################################################    
    
    recording = False
    output_section_rf = []
    for line in open('myfile1.txt').readlines():

        if recording is False:
            if re.search(r'ITEM [0-9]. Risk Factors', line) is not None:
                recording = True
                output_section_rf.append(line.decode('utf-8').strip())
        elif recording is True:
            if re.search(r'ITEM [0-9].', line) is not None:
                recording = False
            output_section_rf.append(line.decode('utf-8').strip())
    str1 = ''.join(output_section_rf)

    
    variable_names = main_function(str1)


    k="rf"
    length=0
    for j in variables:
        col = k.lower() + '_' + j
        df2.loc[row][col] = variable_names[length]
        length+=1
        
##############################################################################################        
    output_section_whole = [] 
    for line in open('myfile1.txt').readlines():
        output_section_whole.append(line.decode('utf-8').strip())
    str1 = ''.join(output_section_whole)
    sent_list = word_tokenize(str1)
    sent_list = remove_stopwords(sent_list)
    sent_list = normalize_making(sent_list)
    
    constraining_whole.loc[row] = 0
    constraining_words_whole_report_count = 0
    for word in sent_list:
        
        if word in constraining:
            constraining_words_whole_report_count += 1
   
    constraining_whole.loc[row] = constraining_words_whole_report_count
    
####################################################################################################


df = pd.concat([df1,df2,constraining_whole], axis = 1)
# df.shape    
    
# print(df)   




   


 


[nltk_data] Downloading package wordnet to /Users/apple/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151


In [11]:
writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer,'Sheet1')